In [169]:
#pip3 install clean-fid numba numpy torch==2.0.0+cu118 torchvision --force-reinstall --extra-index-url https://download.pytorch.org/whl/cu118
#pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118html
#pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
# check torch, torch-sparse, torch-scatter, pytorch_geometric versions

import random
import numpy as np

from torch import nn
import torch

import torch.nn.functional as F
from torch_geometric.data import Data

import torch_geometric.transforms as T
from torch_geometric.nn import SGConv
import pickle

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

import os
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [170]:
os.getcwd()

'/home/dhruvsinha/DePaul-Reproducible-ML/running_pipeline'

### Graphsage Architecture

In [171]:
class GraphSAGE(torch.nn.Module):

    """
    input dimension: dimension of the feature vector
    output dimension: dimension of the node (this should be equal to the dmension of the trace)
    """
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, out_dim)
    
    def forward(self, data):
        x, adj_t = data.x, data.edge_index
        x = self.conv1(x, adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv3(x, adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout)
        #return torch.log_softmax(x, dim=-1)
        #print("output graphsage", x)
        return x

### MLP Architecture

In [172]:
class MLP(torch.nn.Module):
    """
    This will the one-hot encoded labels of all the nodes in the graph and output 
    a output_dim long vector

    input dim: vocab size
    output dim: 100 (a hyperparameter)
    """
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 200)
        self.hidden_fc = nn.Linear(200,150)
        self.output_fc = nn.Linear(150, output_dim)

    def forward(self, x):
        # x = [num_nodes,vocab_size] (412, 412)
        h_1 = F.relu(self.input_fc(x))
        # h_1 = [num_nodes, 200] (412, 200)

        h_2 = F.relu(self.hidden_fc(h_1))
        # h_2 = [num_nodes, 150] (412, 150)

        output_mlp = self.output_fc(h_2)
        #output_mlp = [num_nodes, output_dim] (412, 100)

        return output_mlp

### LSTM Architecture

In [173]:
"""
class lstm(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size):
		super(lstm, self).__init__()
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.lstm = nn.LSTM(vocab_size, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)


	def forward(self, input_sentence, batch_size=None):
		input = input_sentence.permute(1, 0, 2) 
		h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size))
		c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size))
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
		final_output = self.label(output) 
		#print("final_output from lstm", final_output)
		return (final_output)
"""

class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_dim):
        super(BRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_dim = output_dim

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True,
                            bidirectional = True)
        self.fc = nn.Linear(hidden_size*2, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)

        out, (hidden_state, cell_state) = self.lstm(x,(h0,c0))
        #print("output shape inside lstm", out.shape)
        out = self.fc(out)
        return out

### Training

In [174]:
def train(model_mlp, optimizer_mlp, 
        model_graphsage, optimizer_graphsage,
        model_lstm, optimizer_lstm,
        data_nodeLabels, data_adjacencyMatrix):
    

    model_lstm.train()
    it = iter(train_iter)
    accuracy_batch_list = []
    loss_batch_list = []
    for i, batch in enumerate(it):

        model_mlp.train()
        optimizer_mlp.zero_grad()
        out_mlp = model_mlp(data_nodeLabels)
        
        # create a graphsage custom data
        data = Data(x=out_mlp, edge_index=adj_2d.t().contiguous())

        #call graphsage
        model_graphsage.train()
        optimizer_graphsage.zero_grad()
        out_graphsage = model_graphsage(data)

        model_lstm.train()
        optimizer_lstm.zero_grad()
        output_lstm = model_lstm(batch)
        #print("shape of output from lstm", output_lstm.shape)
        output_lstm = torch.reshape(output_lstm,(batch_size, trace_length, output_dim_graphsage)) #reshape it to batch_lenght, seq_length, output_dim

        loss_calc, result = calculate_loss(output_lstm, out_graphsage, batch)
        accuracy = evaluate_training(result, batch)
        accuracy_batch_list.append(accuracy)
        loss_batch_list.append(loss_calc.item())
        #print("loss", loss_calc)
        loss_calc.backward()
        #print("grad of input fc of MLP", model_mlp.input_fc.weight.grad)
        optimizer_graphsage.step()
        #print("grad of input fc of lstm", list(model_lstm.parameters())) 
        optimizer_mlp.step()
        optimizer_lstm.step()
    accuracy_entire_epoch = sum(accuracy_batch_list)/len(accuracy_batch_list)
    loss_entire_epoch = sum(loss_batch_list)/len(loss_batch_list)
    
    return (accuracy_entire_epoch, loss_entire_epoch)

### Loss function

In [175]:
def calculate_loss(output_lstm, out_graphsage, batch):
    # batch here is the true representation. This is because the vectors in batch are 
    # one hot encoded traces
    out_graphsage_reshaped = torch.transpose(out_graphsage, 0, 1)
    dot_product = torch.matmul(output_lstm, out_graphsage_reshaped)
    sm = nn.Softmax(dim=2)
    
    dot_product = sm(dot_product)
    
    loss_calc = loss_fn(dot_product,batch)
    #evaluate_training(c,batch)
    return (loss_calc, dot_product)

### Evaluate training accuracy

In [176]:
def evaluate_training(result, truth):
    
    argmaxed_result = torch.argmax(result, dim=2)
    argmaxed_truth = torch.argmax(truth, dim=2)
    
    number_correctly_mapped = 0
    for i in range(len(argmaxed_result)):
        trace_res =  argmaxed_result[i]
        trace_orig = argmaxed_truth[i]
        if torch.equal(trace_res,trace_orig):
            number_correctly_mapped+= 1
    
    accuracy = number_correctly_mapped/len(argmaxed_result)

    return accuracy
    


### Dataset Preprocessing

#### Converting sparse adjacency matrix to dense

In [177]:
def structure_adjacency_matrix():
    """
    This method is to convert the original adjacency matrix into a 2 dimensional 
    adjacency matrix
    """
    adj_2d = []
    t_ind = 0
    for t in data_adjacencyMatrix:
        elem_ind = 0
        for elem in t:
            if elem == 1:
                a= [t_ind, elem_ind]
                adj_2d.append(a)
            elem_ind += 1
        t_ind = t_ind+1
    return torch.tensor(adj_2d)


#### Import trace data

In [178]:
traces = pickle.load(open("/home/dhruvsinha/DePaul-Reproducible-ML/running_pipeline/traces.pkl", "rb"))
traces = traces.int()
#traces = traces[:9900]
traces.shape


torch.Size([1000, 6])

#### Import nodes and labels

In [179]:
data_nodeLabels = pickle.load(open("/home/dhruvsinha/DePaul-Reproducible-ML/running_pipeline/nodeLabels.pkl", "rb"))
data_adjacencyMatrix_old = pickle.load(open("/home/dhruvsinha/DePaul-Reproducible-ML/running_pipeline/adjacencyMatrix.pkl", "rb"))

data_adjacencyMatrix = data_adjacencyMatrix_old.type(torch.int64)


num_nodes_graph = data_adjacencyMatrix.shape[0]

#### Converting traces to OHE

In [180]:
"""
traces = [[0,1,4,15,38],
[0,1,6,18,40],
[0,1,4,15,38],
[0,1,6,18,40]]
"""
all_traces_ohe = []
for t in traces:
    trace_ohe = []
    for t_node in t:
        node_pos = [0]*num_nodes_graph
        #node_pos.append(t_node)# attaching the node label
        node_pos[t_node] = 1
        trace_ohe.append(node_pos)
    all_traces_ohe.append(trace_ohe)

all_data= torch.FloatTensor(all_traces_ohe)
all_data.shape

torch.Size([1000, 6, 1365])

In [181]:
train_data = all_data[:len(all_data)-100]
test_data = all_data[len(all_data)-100:]
print("train data shape", train_data.shape)
print("test data shape", test_data.shape)

train data shape torch.Size([900, 6, 1365])
test data shape torch.Size([100, 6, 1365])


### Defining hyperparameters and initiating the model

In [182]:
lr = 1e-4 
epochs = 50
trace_length = traces.shape[1]

in_dim_mlp = data_nodeLabels.shape[1] #current vocab size
output_dim_mlp = 100 # this is the output dimension of mlp (a hyperparameter)

input_dim_graphsage = output_dim_mlp # input dimension of graphsage = output dimension of mlp
hidden_dim_graphsage = 75 #this is a hyperparameter
output_dim_graphsage = 50 #this is a hyperparameter

model_mlp = MLP(input_dim=in_dim_mlp, 
                 output_dim= output_dim_mlp)
optimizer_mlp = torch.optim.Adam(model_mlp.parameters(), lr=lr)


model_graphsage = GraphSAGE(in_dim=input_dim_graphsage, 
                 hidden_dim=hidden_dim_graphsage, 
                 out_dim= output_dim_graphsage)
optimizer_graphsage = torch.optim.Adam(model_graphsage.parameters(), lr=lr)

vocab_size = in_dim_mlp
batch_size = 50
output_size_lstm = output_dim_graphsage
hidden_size_lstm = 100
train_iter = DataLoader((train_data), batch_size = batch_size)

num_layers_lstm = 2
input_size_lstm = vocab_size
model_lstm = BRNN(input_size_lstm, hidden_size_lstm, num_layers_lstm, output_size_lstm)
#model_lstm = BRNN(batch_size, output_size_lstm, hidden_size_lstm, vocab_size)
optimizer_lstm = torch.optim.Adam(model_lstm.parameters(), lr=lr)

adj_2d = structure_adjacency_matrix()

loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(1, 1 + epochs):
    accuracy_entire_epoch, loss_entire_epoch = train(model_mlp, optimizer_mlp, 
        model_graphsage, optimizer_graphsage,
        model_lstm, optimizer_lstm,
        data_nodeLabels, adj_2d)
    
    print("training accuracy for epoch {} = ".format(epoch), accuracy_entire_epoch)
    print("training loss for epoch {} = ".format(epoch), loss_entire_epoch)

print("Process complete")

training accuracy for epoch 1 =  0.0
training loss for epoch 1 =  0.007875865635772547
training accuracy for epoch 2 =  0.0
training loss for epoch 2 =  0.007875864963150687
training accuracy for epoch 3 =  0.0
training loss for epoch 3 =  0.007875865118371116
training accuracy for epoch 4 =  0.0
training loss for epoch 4 =  0.007875864704449972
training accuracy for epoch 5 =  0.0
training loss for epoch 5 =  0.007875864549229542
training accuracy for epoch 6 =  0.0
training loss for epoch 6 =  0.00787586434226897
training accuracy for epoch 7 =  0.0
training loss for epoch 7 =  0.007875864187048541
training accuracy for epoch 8 =  0.0
training loss for epoch 8 =  0.007875863617906967
training accuracy for epoch 9 =  0.0
training loss for epoch 9 =  0.007875863617906967
training accuracy for epoch 10 =  0.0
training loss for epoch 10 =  0.007875863773127397
training accuracy for epoch 11 =  0.0
training loss for epoch 11 =  0.007875863410946395
training accuracy for epoch 12 =  0.0
tr

In [183]:
def test_model(data_to_test):
    model_mlp.eval()
    with torch.no_grad():
        out_mlp = model_mlp(data_nodeLabels)

    data = Data(x=out_mlp, edge_index=adj_2d.t().contiguous())

    model_graphsage.eval()
    with torch.no_grad():
        graphsage_output = model_graphsage(data)

    model_lstm.eval()
    with torch.no_grad():
        lstm_test_result = model_lstm(data_to_test)

    accuracy = evaluate_test(lstm_test_result, data_to_test)
    return accuracy



def evaluate_test(result, truth):
    
    argmaxed_result = torch.argmax(result, dim=2)
    argmaxed_truth = torch.argmax(truth, dim=2)
    
    number_correctly_mapped = 0
    for i in range(len(argmaxed_result)):
        trace_res =  argmaxed_result[i]
        trace_orig = argmaxed_truth[i]
        if torch.equal(trace_res,trace_orig):
            number_correctly_mapped+= 1
    
    accuracy = number_correctly_mapped/len(argmaxed_result)

    return accuracy

# testing on train data again
test_accuracy = test_model(train_data)

print("Test Accuaracy is ", test_accuaracy)
    

Test Accuaracy is  0.0
